In [7]:
%pip install -qU langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite langchain[openai]

Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults
import dotenv
dotenv.load_dotenv()
from langchain.chat_models import init_chat_model
import getpass
import os
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver


In [14]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")

tools = [search]


In [9]:

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
  
model = init_chat_model("gpt-4", model_provider="openai")

In [13]:
response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [15]:
model_with_tools = model.bind_tools(tools)

In [16]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [17]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_bg5ML1SIViDHK8IHt6FjidZF', 'type': 'tool_call'}]


In [20]:
agent_executor = create_react_agent(model, tools)

response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='5becd00c-4c4d-4582-97b8-5c9fed5f238f'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BLYJMDV5MnzLUmUEjrc6VxbNO3pXB', 'finish_reason': 'stop', 'logprobs': None}, id='run-a29e65cf-66da-41bd-a00f-0c52c2c11dca-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [21]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='c030a7f5-5d6e-4844-a26f-c7014f3b5d97'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kOxNLquxf8t6nVpIExgA81hN', 'function': {'arguments': '{\n  "query": "current weather in san francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 88, 'total_tokens': 113, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BLYK9Z2PcQwMgtFIFMpwK992skrci', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1ca021ee-63dc-47ef-9104-7ab180b4fc05-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in san f

In [22]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_XW8d0eWceGDdZnGPkTvliFrR)
 Call ID: call_XW8d0eWceGDdZnGPkTvliFrR
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1744474858, 'localtime': '2025-04-12 09:20'}, 'current': {'last_updated_epoch': 1744474500, 'last_updated': '2025-04-12 09:15', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/d

In [23]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| San| Francisco|,| California| is| partly| cloudy| with| a| temperature| of| |11|.|7|°C| (|53|.|1|°F|).| The| wind| is| coming| from| the| southwest| at| a| speed| of| |5|.|0| k|ph| (|3|.|1| mph|).| The| humidity| level| is| |71|%| and| the| visibility| is| |16|.|0| km|.| [|Source|](|https|://|www|.weather|api|.com|/)|

In [26]:
memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [27]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 85, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BLYMWVbnzARJY9jCHcd8zZtAZjwH1', 'finish_reason': 'stop', 'logprobs': None}, id='run-39f126c4-d813-4ca9-bd58-824bed7e59e4-0', usage_metadata={'input_tokens': 85, 'output_tokens': 12, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [28]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 108, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BLYMjpdKJ4u9ss4NAVZL0nIobq9w8', 'finish_reason': 'stop', 'logprobs': None}, id='run-bcf4cadd-c102-4276-8652-b0228f745557-0', usage_metadata={'input_tokens': 108, 'output_tokens': 7, 'total_tokens': 115, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [29]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, {"configurable": {"thread_id": "abc124"}}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 86, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-BLYNH7AtIPFJSfxcPUjIBS2mTrtCI', 'finish_reason': 'stop', 'logprobs': None}, id='run-256d8a2f-c3e0-44bd-96dc-4bb1d74b1d8f-0', usage_metadata={'input_tokens': 86, 'output_tokens': 40, 'total_tokens': 126, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
